In [4]:
# python -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
import torch 
import torch.nn as nn 
import torch.nn.functional as F

In [5]:
class SelfAttention(nn.Module): 
    def __init__(self, d_model: int = 2, row_dim: int = 0, col_dim: int = 1, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)  
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False) 
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False) 

        self.row_dim = row_dim 
        self.col_dim = col_dim 
    
    def forward(self, token_encodings):  
        q = self.W_q(token_encodings) 
        k = self.W_k(token_encodings) 
        v = self.W_v(token_encodings)

        similarity_scores = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim)) 
        scaled_sims = similarity_scores / torch.tensor(k.size(self.col_dim) ** 0.5)  
        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)
        attention_scores = torch.matmul(attention_percents, v) 
        return attention_scores   
    
 

In [6]:
# sample matrix for encodings 
encodings_matrix = torch.tensor([
    [1.16, 0.23], 
    [0.57, 1.36], 
    [4.41, -2.16]
])

torch.manual_seed(42) 

model = SelfAttention(
    d_model=2, 
    row_dim=0, 
    col_dim=1
)

model(encodings_matrix) 

tensor([[1.0100, 1.0641],
        [0.2040, 0.7057],
        [3.4989, 2.2427]], grad_fn=<MmBackward0>)

In [8]:
model.W_q.weight.transpose(0, 1) 

tensor([[ 0.5406, -0.1657],
        [ 0.5869,  0.6496]], grad_fn=<TransposeBackward0>)

In [9]:
model.W_k.weight.transpose(0, 1) 

tensor([[-0.1549, -0.3443],
        [ 0.1427,  0.4153]], grad_fn=<TransposeBackward0>)

In [10]:
model.W_v.weight.transpose(0, 1) 

tensor([[ 0.6233,  0.6146],
        [-0.5188,  0.1323]], grad_fn=<TransposeBackward0>)

In [11]:
model.W_q(encodings_matrix)

tensor([[ 0.7621, -0.0428],
        [ 1.1063,  0.7890],
        [ 1.1164, -2.1336]], grad_fn=<MmBackward0>)

In [12]:
model.W_k(encodings_matrix)

tensor([[-0.1469, -0.3038],
        [ 0.1057,  0.3685],
        [-0.9914, -2.4152]], grad_fn=<MmBackward0>)

In [13]:
model.W_v(encodings_matrix)

tensor([[ 0.6038,  0.7434],
        [-0.3502,  0.5303],
        [ 3.8695,  2.4246]], grad_fn=<MmBackward0>)

In [14]:
q = model.W_q(encodings_matrix) 
q

tensor([[ 0.7621, -0.0428],
        [ 1.1063,  0.7890],
        [ 1.1164, -2.1336]], grad_fn=<MmBackward0>)

In [15]:
k = model.W_k(encodings_matrix) 
k 

tensor([[-0.1469, -0.3038],
        [ 0.1057,  0.3685],
        [-0.9914, -2.4152]], grad_fn=<MmBackward0>)

In [16]:
v = model.W_v(encodings_matrix) 
v 

tensor([[ 0.6038,  0.7434],
        [-0.3502,  0.5303],
        [ 3.8695,  2.4246]], grad_fn=<MmBackward0>)

In [17]:
sims = torch.matmul(q, k.transpose(dim0=0, dim1=1)) 
sims

tensor([[-0.0990,  0.0648, -0.6523],
        [-0.4022,  0.4078, -3.0024],
        [ 0.4842, -0.6683,  4.0461]], grad_fn=<MmBackward0>)

In [18]:
scaled_sims = sims / (torch.tensor(2) ** 0.5) 
scaled_sims 

tensor([[-0.0700,  0.0458, -0.4612],
        [-0.2844,  0.2883, -2.1230],
        [ 0.3424, -0.4725,  2.8610]], grad_fn=<DivBackward0>)

In [19]:
attention_percents = F.softmax(scaled_sims, dim=1) 
attention_percents

tensor([[0.3573, 0.4011, 0.2416],
        [0.3410, 0.6047, 0.0542],
        [0.0722, 0.0320, 0.8959]], grad_fn=<SoftmaxBackward0>)

In [20]:
torch.matmul(attention_percents, model.W_v(encodings_matrix)) 

tensor([[1.0100, 1.0641],
        [0.2040, 0.7057],
        [3.4989, 2.2427]], grad_fn=<MmBackward0>)